In [22]:
import pandas as pd
import numpy as np

from sklearn.model_selection import cross_val_score, KFold, train_test_split
from sklearn.model_selection import LeaveOneOut
from sklearn.preprocessing import PolynomialFeatures

from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn import tree
from sklearn.svm import SVR
import xgboost as xgb
from sklearn.linear_model import Lasso
from sklearn.neural_network import MLPRegressor

from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_percentage_error

In [23]:
df = pd.read_csv('IphoneData.csv')
newdf = df.drop(columns=['NAME', 'CO2E'])
features = newdf.to_numpy()
emissions = df['CO2E'].to_numpy()

In [35]:
def modelScore(X, y, model):
    avgSSE = []
    avgMAPE = []
    splits = LeaveOneOut().get_n_splits(X)
    #cross validation using leave one out
    for i, (trainIndex, testIndex) in enumerate(LeaveOneOut().split(X)):
        X_train = X[trainIndex]
        X_test = X[testIndex]
        y_train = y[trainIndex]
        y_test = y[testIndex]
    

        if model == 'linear regression':
            fittedModel = LinearRegression().fit(X_train, y_train)
                
        elif model == 'decision tree':
            fittedModel = tree.DecisionTreeRegressor().fit(X_train, y_train)
                
        elif model == 'random forest':
            y_train = y_train.flatten()
            RF = RandomForestRegressor()#ccp_alpha=2.6, max_samples=.9, max_features='log2', min_samples_split=2,
                      #n_estimators=50, max_depth=17, min_samples_leaf=1, criterion='squared_error')
            fittedModel = RF.fit(X_train, y_train)
            
        elif model == 'support vector regression':
            y_train = y_train.flatten()
            fittedModel = SVR(kernel = 'linear').fit(X_train, y_train)
    
        elif model == 'polynomial regression':
            polyFeatures = PolynomialFeatures(degree=3, interaction_only=True, include_bias=True, order='C')
            polyX_train = polyFeatures.fit_transform(X_train)
            polyX_test = polyFeatures.transform(X_test)
            fittedModel = LinearRegression().fit(polyX_train, y_train)
            X_test = polyX_test

        elif model == 'xgboost':
            xgbModel = xgb.XGBRegressor(learning_rate=0.225, n_estimators=225, booster='gblinear',
                                        reg_alpha=0, reg_lambda=0, eval_metric='auc', 
                                        objective='reg:squaredlogerror', updater='coord_descent', 
                                        feature_selector='thrifty')
            fittedModel = xgbModel.fit(X_train, y_train)
        
        elif model =='lasso':
            lassoModel = Lasso(tol=.0001, max_iter=2000, selection='random', alpha=.15)
            fittedModel = lassoModel.fit(X_train, y_train)

        elif model =="neural network":
            nn = MLPRegressor(alpha=0.001, batch_size=25, hidden_layer_sizes=(10, 10, 10, 10),
             learning_rate_init=0.05, max_iter=10000, random_state=42, tol=1e-1, max_fun=25000,
             solver='lbfgs', activation='identity')
            
            fittedModel = nn.fit(X_train, y_train)          

        y_pred = fittedModel.predict(X_test)
        
        #scores
        avgSSE.append(np.sum((y_test - y_pred) ** 2))
        avgMAPE.append(mean_absolute_percentage_error(y_test, y_pred))

    avgSSE, avgMAPE = np.mean(avgSSE), np.mean(avgMAPE)

   #print results
    """print(model)
    print("SSE = " + str(avgSSE))
    print("MAPE = " + str(avgMAPE))"""
    print()
    print(str(avgSSE))
    print(str(avgMAPE))



"""alpha=0.1, batch_size=25, hidden_layer_sizes=(10, 10, 10, 10),
             learning_rate_init=0.5, max_iter=12000, random_state=42, tol=1.01, max_fun=15000,
             solver='lbfgs', activation='identity'"""


"alpha=0.1, batch_size=25, hidden_layer_sizes=(10, 10, 10, 10),\n             learning_rate_init=0.5, max_iter=12000, random_state=42, tol=1.01, max_fun=15000,\n             solver='lbfgs', activation='identity'"

In [36]:
modelScore(features, emissions, 'neural network')

KeyboardInterrupt: 

In [ ]:
modelScore(features, emissions, 'linear regression')

SSE = 19.697207782900612
MAPE = 0.04873289978762851


In [ ]:
modelScore(features, emissions, 'decision tree')

SSE = 59.270833333333336
MAPE = 0.08236857208050594


In [ ]:
modelScore(features, emissions, 'random forest')

SSE = 39.72454166666668
MAPE = 0.0661206060152571


In [ ]:
modelScore(features, emissions, 'support vector regression')

SSE = 23.649005892460583
MAPE = 0.04674170011550882


In [ ]:
modelScore(features, emissions, 'polynomial regression')

SSE = 6.48902173028647
MAPE = 0.02359235125487665


In [ ]:
modelScore(features, emissions, 'xgboost')

SSE = 19.403605405051774
MAPE = 0.04430870011778093


In [ ]:
modelScore(features, emissions, 'lasso')

SSE = 20.55092733111427
MAPE = 0.047764169916753564


In [38]:
modelsList = ['linear regression', 'decision tree', 'random forest', 'support vector regression', 'polynomial regression', 'xgboost', 'lasso', 'neural network']
multipleFeatures = df[['DISPLAY (inch)', 'STORAGE (gb)', 'MEMORY (gb)', 'GPU (cores)', 'NEURAL ENGINE (cores)', 'WEIGHT (gm)', 'FRONT CAMERA (mp)', 'Sum of megapixels', 'MAIN CAMERA (mp)']].to_numpy()
featuresList = ['DISPLAY (inch)', 'STO|RAGE (gb)', 'MEMORY (gb)', 'CPU (cores)', 'GPU (cores)', 'NEURAL ENGINE (cores)', 'WEIGHT (gm)', 'FRONT CAMERA (mp)', 'BATTERY (mAh)', 'number of camera (back)', 'Sum of megapixels', 'MAIN CAMERA (mp)']

In [39]:
for model in modelsList:
    modelScore(multipleFeatures, emissions, model)


53.50968646482113
0.07806507451931323

90.375
0.08808646011475191

39.38223333333334
0.06470724876118629

62.62413071130815
0.07569110098334662

214.53039509254018
0.0994135834939639

56.038613827849076
0.07258845065163709

53.938458338363176
0.07860601561757914

53.50971633791564
0.07805246714092394
